In [ ]:
!pip  install langchain langchain-community langchainhub chromadb bs4 sentence_transformers pypdf

In [ ]:
!pip install langchain
!pip install openai
!pip install langchain-huggingface
!pip install huggingface_hub
!pip install transformers
!pip install accelerate
!pip install  bitsandbytes
!pip install langchain-community langchain-core
!pip install --upgrade langchain


In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import BSHTMLLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate
import pandas as pd
import string
from langchain.docstore.document import Document
import os
from langchain_huggingface import HuggingFacePipeline
from langchain_huggingface import HuggingFaceEndpoint
import re
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
import torch

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig, pipeline
from huggingface_hub import login

In [ ]:
sec_key = ""
os.environ["HUGGINGFACEHUB_API_TOKEN"]=sec_key


In [ ]:
path = "/content/max_life_online_term_plan_plus_prospectus_72e220c5b0.pdf"

In [ ]:
Chuck_Size = 1200
Chunk_Overlap = 0

In [ ]:
loader = PyPDFLoader(path )
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=Chuck_Size, chunk_overlap=Chunk_Overlap, add_start_index=True)

splits = text_splitter.split_documents(docs)

In [ ]:
type(splits)

list

In [ ]:
splits[0]

Document(metadata={'source': '/content/max_life_online_term_plan_plus_prospectus_72e220c5b0.pdf', 'page': 0, 'start_index': 9}, page_content='Page 1 of 25 \n \n \nMax Life Online Term Plan Plus \nA Non-Linked Non-Participating Individual Pure Risk Premium Life Insurance Plan \nUIN: 104N092V04 \n \nPROSPECTUS \n \nLife Insurance Coverage is available in this Product. \n \nAbout Max Life Insurance \n  \nMax Life Insurance Company Limited (“Max Life”) is a Joint Venture between Max Financial Services Limited and Axis \nBank Limited. Max Financial Services Ltd. is a part of the Max Group, an Indian multi -business corporation. \n \nMax Life offers comprehensive protection and long -term savings life insurance solutions, through its multichannel distribution \nincluding agency and third-party distribution partners. Max Life has built its operations over almost two decades through need -\nbased sales process, a customer-centric approach to engagement and service delivery and trained human ca

In [ ]:
splits[1]

Document(metadata={'source': '/content/max_life_online_term_plan_plus_prospectus_72e220c5b0.pdf', 'page': 0, 'start_index': 1125}, page_content="For more information, please visit the Company's website at www.maxlifeinsurance.com \n \nMax Life Online Term Plan Plus \n \nYou have always strived to give your family the best in life and have ensured they fulfill their dreams. However, in your heart, \nyou always feel insecure about their future in your absence. Will your family be able to sustain the same lifestyle even in y our \nabsence? \n \nTo put all your fears to rest and to provide you with  peace of mind, Max Life Insurance offers Max Life Online Term Plan \nPlus. A comprehensive protection solution for your loved ones at an affordable price. Not only that, you can now avoid the \nhassle of tedious paperwork and easily get yourself insured online. \n \nKey features of Max Life Online Term Plan Plus: \n \n• Choice of 3 death benefit options: Max Life Online Term Plan Plus helps you

In [ ]:
texts = [s.page_content for s in splits]

In [ ]:
df = {"text" : texts }
df = pd.DataFrame(df)

In [ ]:
df.to_csv("sample-data.csv", index = False)

In [ ]:
repo_id="mistralai/Mistral-7B-Instruct-v0.2"

llm=HuggingFaceEndpoint(repo_id=repo_id,max_length=2048,temperature=0.1,token=sec_key)



                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.


In [ ]:

def gen_prompt(input_text):

    prompt_template = f"""
    You are a legal expert analyzing policy documents. Classify the following text into the most appropriate categories listed below. Return **only the applicable categories**, and ensure that no explanation is included—just return the list of categories. If none apply, return an empty list `[]`.

    ### Categories and Examples:
    1. **Conditional Clauses**: Conditional clauses state that an outcome, benefit, or action depends on the fulfillment of specific conditions. When multiple conditions are implied or not explicitly defined, the sentence can become complex, unclear, or open to interpretation, increasing the risk of misinterpretation.
    - Example: "Currently, the Special Surrender Value (SSV) is the same as the Guaranteed Surrender Value. The SSV may be revised from time to time with prior approval of the Authority."

    2. **Cross-Dependent Clauses**: These refer to clauses that rely on references to other sections of the document for full understanding. Cross-references often require readers to navigate multiple sections, making the content harder to follow and increasing the risk of misinterpretation.
    - Example: "Coverage for losses arising from natural disasters as defined under Section 2.1 shall only apply if the policyholder has fulfilled the obligations specified in Section 3.4, including the payment of additional premiums detailed in Appendix A."

    3. **Legal Terminology**: These sentences are challenging to understand due to intricate phrasing or technical legal vocabulary. Such complexity can blur the intended meaning, increasing the likelihood of misinterpretation.
    - Example: "In accordance with the provisions outlined in this agreement, any claims for reimbursement submitted by the policyholder must be accompanied by all requisite documentation, including, but not limited to, certified copies of original receipts, detailed proof of loss forms, and corroborating evidence from third-party service providers, failing which the insurer reserves the right, at its sole discretion, to deny said claims without further obligation to notify the policyholder of deficiencies."

    4. **Ambiguity in Expression**: This class refers to sentences or clauses in policy documents that are unclear or open to multiple interpretations due to vague or imprecise language. Examples include:
    - **Vague Terms**: Words like "some", "many", or "a few," which lack precise meaning and can be interpreted differently by different readers.
    - **Modal Verbs**: Words like "may," "could," "might," or "should" that introduce uncertainty and flexibility in the interpretation of the statement.
    - Example: "The refund will be processed soon, but it may take a while."

    ### Instructions:
    - First, identify key linguistic indicators that belong to each category (such as conditionals like "may," "could," "shall"; cross-references like "Section 2.1," "Appendix A"; or complex legal terms like "provisions" and "corroborating evidence").
    - Assess if the sentence introduces uncertainty, uses vague terms, or provides unclear definitions—any of these would signal potential ambiguity or conditionality.
    - Think about whether parts of the text reference other sections, creating a need to follow up for understanding. If such references are crucial for full comprehension, the "Cross-Dependent Clauses" category is likely applicable.

    ### Text for Classification:
    {input_text}

    ### Output Format:
    - Only return a list of the applicable categories in the exact format below. Do not add additional text.
    - `["Category1", "Category2"]`
    - If no categories apply, return `[]`.
    """
    return prompt_template

In [ ]:

def extract(input_string):

    match = re.search(r'\[([^\]]*)\]', input_string)

    if match:
        # Extract the matched list (including its contents)
        extracted_list = match.group(0)



        # If you want to extract just the contents of the list
        extracted_content = match.group(1).split(',') if match.group(1) else []  # Handle empty list

        # Clean up the elements, stripping spaces and quotes
        extracted_content = [item.strip().strip('"') for item in extracted_content]

        return extracted_content


    else:

        return []

In [ ]:
df.head(1)

,text
0,Page 1 of 25 \n \n \nMax Life Online Term Plan...


In [ ]:
text = list(df["text"])



# Only using local context

In [ ]:
T = []
res1 = []
for t in tqdm(text):

    t2 = gen_prompt(t)

    ans = llm.invoke(t2)
    ans = extract(ans)

    res1.append(ans)
    T.append(t)


100%|██████████| 73/73 [01:44<00:00,  1.43s/it]


In [ ]:
res1

[[''],
 [],
 [],
 ['Conditional Clauses'],
 ['Conditional Clauses'],
 [],
 ['Conditional Clauses'],
 ['Conditional Clauses'],
 [],
 ['Conditional Clauses', 'Legal Terminology'],
 ['Conditional Clauses'],
 ['Conditional Clauses'],
 ['Conditional Clauses', 'Ambiguity in Expression'],
 ['Conditional Clauses', 'Ambiguity in Expression'],
 [],
 ['Conditional Clauses'],
 ['Conditional Clauses'],
 [],
 ['Minimum Age: 18 years" and "Maximum Age: 85 years" are fixed values',
  'so no conditionality or ambiguity is present.'],
 ['Conditional Clauses'],
 ['Conditional Clauses'],
 [],
 ['Legal Terminology'],
 [],
 ['Conditional Clauses'],
 ['Conditional Clauses'],
 [],
 ['Conditional Clauses', 'Ambiguity in Expression'],
 ['Conditional Clauses', 'Cross-Dependent Clauses', 'Legal Terminology'],
 ['Conditional Clauses'],
 ['Conditional Clauses', 'Legal Terminology', 'Ambiguity in Expression'],
 ['Conditional Clauses'],
 ['Legal Terminology'],
 ['Conditional Clauses', 'Legal Terminology'],
 ['Conditi

In [ ]:
seen = ["Conditional Clauses", "Cross-Dependent Clauses", "Legal Terminology", "Ambiguity in Expression"]

T_fil = []
res_fil = []
for i, j in zip(res1, T):

    if(len(i) > 0):

        if all(value in seen for value in i):

            res_fil.append(i)
            T_fil.append(j)

    else:

        res_fil.append(i)
        T_fil.append(j)

In [ ]:
res_fil

[[],
 [],
 ['Conditional Clauses'],
 ['Conditional Clauses'],
 [],
 ['Conditional Clauses'],
 ['Conditional Clauses'],
 [],
 ['Conditional Clauses', 'Legal Terminology'],
 ['Conditional Clauses'],
 ['Conditional Clauses'],
 ['Conditional Clauses', 'Ambiguity in Expression'],
 ['Conditional Clauses', 'Ambiguity in Expression'],
 [],
 ['Conditional Clauses'],
 ['Conditional Clauses'],
 [],
 ['Conditional Clauses'],
 ['Conditional Clauses'],
 [],
 ['Legal Terminology'],
 [],
 ['Conditional Clauses'],
 ['Conditional Clauses'],
 [],
 ['Conditional Clauses', 'Ambiguity in Expression'],
 ['Conditional Clauses', 'Cross-Dependent Clauses', 'Legal Terminology'],
 ['Conditional Clauses'],
 ['Conditional Clauses', 'Legal Terminology', 'Ambiguity in Expression'],
 ['Conditional Clauses'],
 ['Legal Terminology'],
 ['Conditional Clauses', 'Legal Terminology'],
 ['Conditional Clauses'],
 ['Conditional Clauses'],
 ['Conditional Clauses'],
 ['Ambiguity in Expression'],
 ['Conditional Clauses'],
 ['Condi

In [ ]:
df_temp = {"text" : T_fil , "categories" : res_fil}
df_temp = pd.DataFrame(df_temp)

df_temp.head(10)

,text,categories
0,"For more information, please visit the Company...",[]
1,• Accelerated payout in case diagnosed with 40...,[]
2,Page 2 of 25 \n \nStep 3: Upload relevant docu...,[Conditional Clauses]
3,including medical examination report of the Li...,[Conditional Clauses]
4,death of Life Insured. \n \nPlus \n \nIncreasi...,[]
5,Page 3 of 25 \n \nfirst year monthly income sh...,[Conditional Clauses]
6,2) The monthly income will be payable each ...,[Conditional Clauses]
7,2) Accelerated Critical Illness (CI) Benefit,[]
8,Page 4 of 25 \n \nYou may choose a Sum Assured...,"[Conditional Clauses, Legal Terminology]"
9,• The CI Sum Assured will always be paid as a ...,[Conditional Clauses]


In [ ]:

# rows = []
# for text, labels in zip(T_fil, res_fil):
#     # Create a row with 1 or 0 based on the presence of categories
#     row = {category: int(category in labels) for category in seen}
#     row["text"] = text
#     row["Interpretable"] = int(bool(labels))  # 1 if there's any label, else 0
#     rows.append(row)


# df = pd.DataFrame(rows, columns=["text"] + seen + ["Interpretable"])



In [ ]:

categories = ["Conditional Clauses", "Cross-Dependent Clauses", "Legal Terminology", "Ambiguity in Expression"]


data = []
for text, res in zip(T_fil, res_fil):
    row = {category: (1 if category in res else 0) for category in categories}
    row["text"] = text
    row["Interpretable"] = 1 if not res else 0  # Set 'Interpretable' to 1 if res is empty
    data.append(row)


df = pd.DataFrame(data)
df = df[["text"] + categories + ["Interpretable"]]





In [ ]:
df.head(10)

,text,Conditional Clauses,Cross-Dependent Clauses,Legal Terminology,Ambiguity in Expression,Interpretable
0,"For more information, please visit the Company...",0,0,0,0,1
1,• Accelerated payout in case diagnosed with 40...,0,0,0,0,1
2,Page 2 of 25 \n \nStep 3: Upload relevant docu...,1,0,0,0,0
3,including medical examination report of the Li...,1,0,0,0,0
4,death of Life Insured. \n \nPlus \n \nIncreasi...,0,0,0,0,1
5,Page 3 of 25 \n \nfirst year monthly income sh...,1,0,0,0,0
6,2) The monthly income will be payable each ...,1,0,0,0,0
7,2) Accelerated Critical Illness (CI) Benefit,0,0,0,0,1
8,Page 4 of 25 \n \nYou may choose a Sum Assured...,1,0,1,0,0
9,• The CI Sum Assured will always be paid as a ...,1,0,0,0,0


In [ ]:
df.to_csv("annotation-local-demo.csv", index = False)

phase 2: Global context:


Workflow for Finding Global Context from PDF Chunks
Filter Chunks: Exclude any PDF chunks containing the input_sentence to avoid self-similarity.

Generate Embeddings: Use a pre-trained SentenceTransformer to encode the input_sentence and the remaining chunks (filtered_chunks) into embeddings.

Retrieve Top Context: Calculate cosine similarity between the input_sentence embedding and each chunk embedding. Use torch.topk to select the top-5 most relevant chunks based on similarity scores.

Refine Results: Filter out overly similar chunks to ensure diverse and meaningful global context.

This provides the most contextually relevant chunks for interpreting the input_sentence.

In [ ]:
#again loading the same datasets

df2 = pd.read_csv("sample-data.csv")

In [ ]:
df2.head(2)

,text
0,Page 1 of 25 \n \n \nMax Life Online Term Plan...
1,"For more information, please visit the Company..."


In [ ]:
input_sentences = [t for t in list(df2["text"])]

In [ ]:
len(input_sentences)

73

In [ ]:
DF = pd.read_csv("/content/sample-data.csv")

In [ ]:
DF.head(1)

,text
0,Page 1 of 25 \n \n \nMax Life Online Term Plan...


In [ ]:
pdf_chunks = [t for t in list(DF["text"])]

In [ ]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")

def get_global(input_sentence, top_k = 3):


    filtered_chunks = [chunk for chunk in pdf_chunks if input_sentence not in chunk]


    chunk_embeddings = embedder.encode(filtered_chunks, convert_to_tensor=True)


    input_embedding = embedder.encode(input_sentence, convert_to_tensor=True)


    cosine_scores = util.pytorch_cos_sim(input_embedding, chunk_embeddings)

    top_results = torch.topk(cosine_scores, k=top_k)

    top_chunks = [filtered_chunks[idx] for idx in top_results[1][0]]

    top_chunks = [f"#Context: {i+1} " + top_chunks[i] for i in range(0,top_k)]

    top_chunks = "\n".join(top_chunks)

    return top_chunks




In [ ]:

def gen_prompt2(input_text,global_context):

    prompt_template = f"""
    You are a legal expert analyzing policy documents. Classify the following text into the most appropriate categories listed below, using the given global context as a reference. Return **only the applicable categories**, and ensure that no explanation is included—just return the list of categories. If none apply, return an empty list `[]`.

    ### Categories and Examples:
    1. **Conditional Clauses**: Conditional clauses state that an outcome, benefit, or action depends on the fulfillment of specific conditions. When multiple conditions are implied or not explicitly defined, the sentence can become complex, unclear, or open to interpretation, increasing the risk of misinterpretation.
        - Example: "Currently, the Special Surrender Value (SSV) is the same as the Guaranteed Surrender Value. The SSV may be revised from time to time with prior approval of the Authority."

    2. **Cross-Dependent Clauses**: These refer to clauses that rely on references to other sections of the document for full understanding. Cross-references often require readers to navigate multiple sections, making the content harder to follow and increasing the risk of misinterpretation.
        - Example: "Coverage for losses arising from natural disasters as defined under Section 2.1 shall only apply if the policyholder has fulfilled the obligations specified in Section 3.4, including the payment of additional premiums detailed in Appendix A."

    3. **Legal Terminology**: These sentences are challenging to understand due to intricate phrasing or technical legal vocabulary. Such complexity can blur the intended meaning, increasing the likelihood of misinterpretation.
        - Example: "In accordance with the provisions outlined in this agreement, any claims for reimbursement submitted by the policyholder must be accompanied by all requisite documentation, including, but not limited to, certified copies of original receipts, detailed proof of loss forms, and corroborating evidence from third-party service providers, failing which the insurer reserves the right, at its sole discretion, to deny said claims without further obligation to notify the policyholder of deficiencies."

    4. **Ambiguity in Expression**: This class refers to sentences or clauses in policy documents that are unclear or open to multiple interpretations due to vague or imprecise language. Examples include:
        - **Vague Terms**: Words like "some", "many", or "a few," which lack precise meaning and can be interpreted differently by different readers.
        - **Modal Verbs**: Words like "may," "could," "might," or "should" that introduce uncertainty and flexibility in the interpretation of the statement.
        - Example: "The refund will be processed soon, but it may take a while."



    ### Instructions:
    - First, identify key linguistic indicators that belong to each category (such as conditionals like "may," "could," "shall"; cross-references like "Section 2.1," "Appendix A"; or complex legal terms like "provisions" and "corroborating evidence").
    - Assess if the sentence introduces uncertainty, uses vague terms, or provides unclear definitions—any of these would signal potential ambiguity or conditionality.
    - Think about whether parts of the text reference other sections, creating a need to follow up for understanding. If such references are crucial for full comprehension, the "Cross-Dependent Clauses" category is likely applicable.
    - Use the **global context** to help determine which category is most appropriate, especially for sections or terms that require external understanding.

    ### Text for Classification:
    {input_text}


    ### Global Context:
    {global_context}

    ### Output Format:
    - Only return a list of the applicable categories in the exact format below. Do not add additional text.
    - `["Category1", "Category2"]`
    - If no categories apply, return `[]`.
    """
    return prompt_template


In [ ]:
res2 = []

for t in tqdm(input_sentences):

    context = get_global(t)

    input = gen_prompt2(t,context)

    ans = llm.invoke(input)
    ans = extract(ans)

    res2.append(ans)








100%|██████████| 73/73 [13:14<00:00, 10.88s/it]


In [ ]:
res2

[[],
 ['Conditional Clauses',
  'Cross-Dependent Clauses',
  'Legal Terminology',
  'Ambiguity in Expression'],
 ['Conditional Clauses'],
 ['Conditional Clauses', 'Legal Terminology'],
 ['Ambiguity in Expression'],
 ['Conditional Clauses'],
 ['Conditional Clauses'],
 ['Conditional Clauses'],
 ['Conditional Clauses',
  'Cross-Dependent Clauses',
  'Legal Terminology',
  'Ambiguity in Expression'],
 ['Conditional Clauses', 'Legal Terminology'],
 ['Conditional Clauses'],
 ['Conditional Clauses'],
 ['Conditional Clauses', 'Cross-Dependent Clauses'],
 ['Conditional Clauses',
  'Cross-Dependent Clauses',
  'Legal Terminology',
  'Ambiguity in Expression'],
 ['Conditional Clauses'],
 ['Conditional Clauses', 'Cross-Dependent Clauses'],
 ['Conditional Clauses'],
 ['Conditional Clauses'],
 ['Minimum Age: Conditional Clauses',
  'Maximum Age: Conditional Clauses',
  'Minimum Annual Premium: Conditional Clauses'],
 ['Conditional Clauses', 'Legal Terminology'],
 ['Conditional Clauses'],
 ['Conditio

In [ ]:
seen = ["Conditional Clauses", "Cross-Dependent Clauses", "Legal Terminology", "Ambiguity in Expression"]

T_fil2 = []
res_fil2 = []
for i, j in zip(res2, input_sentences):

    if(len(i) > 0):

        if all(value in seen for value in i):

            res_fil2.append(i)
            T_fil2.append(j)

    else:

        res_fil2.append(i)
        T_fil2.append(j)

In [ ]:

categories = ["Conditional Clauses", "Cross-Dependent Clauses", "Legal Terminology", "Ambiguity in Expression"]


data = []
for text, res in zip(T_fil2, res_fil2):
    row = {category: (1 if category in res else 0) for category in categories}
    row["text"] = text
    row["Interpretable"] = 1 if not res else 0  # Set 'Interpretable' to 1 if res is empty
    data.append(row)


df = pd.DataFrame(data)
df = df[["text"] + categories + ["Interpretable"]]





In [ ]:
df.head(10)

,text,Conditional Clauses,Cross-Dependent Clauses,Legal Terminology,Ambiguity in Expression,Interpretable
0,Page 1 of 25 \n \n \nMax Life Online Term Plan...,0,0,0,0,1
1,"For more information, please visit the Company...",1,1,1,1,0
2,• Accelerated payout in case diagnosed with 40...,1,0,0,0,0
3,Page 2 of 25 \n \nStep 3: Upload relevant docu...,1,0,1,0,0
4,including medical examination report of the Li...,0,0,0,1,0
5,death of Life Insured. \n \nPlus \n \nIncreasi...,1,0,0,0,0
6,Page 3 of 25 \n \nfirst year monthly income sh...,1,0,0,0,0
7,2) The monthly income will be payable each ...,1,0,0,0,0
8,2) Accelerated Critical Illness (CI) Benefit,1,1,1,1,0
9,Page 4 of 25 \n \nYou may choose a Sum Assured...,1,0,1,0,0
